In [1]:
import os, json, glob2, pickle
from tqdm import tqdm 

In [2]:
base_dir = "/mnt/BIG-HDD-STORAGE/ebi/arxiv/processed"
triples_dir = f"{base_dir}/triples"
aug_triples_dir = f"{base_dir}/aug_triples_blink"

In [7]:
for counter, item in enumerate(entity_list):
    data_to_link.append({
                            'id': counter,
                            'label': 'unknown',
                            'label_id': -1,
                            'context_left': '', 
                            'mention': item['title'].lower(), 
                            'context_right': item['text'].lower(), 
                        })
_, _, _, _, _, predictions, scores, = main_dense.run(args, None, *models, test_data=data_to_link)

In [8]:
def process_triple(triple_file_dict):
    data_to_link = []
    for sub_section_index, sub_section in enumerate(triple_file_dict):
        if "entities_info" in sub_section:
            for entity_info in sub_section["entities_info"]:
                if entity_info["type"] not in ["Reference", "Code", "Number", "Ost"]:
                    bounds = sorted(entity_info["bounds"], key=lambda item: item[1] - item[0], reverse=True)
                    proper_bound = None
                    for bound in bounds:
                        if ((bound[1] - bound[0]) < 32):
                            proper_bound = bound
                            break
                    if(proper_bound):
                        left_context = ' '.join(entity_info["tokens"][:proper_bound[0]])
                        mention = ' '.join(entity_info["tokens"][proper_bound[0]: proper_bound[1]])
                        right_context = ' '.join(entity_info["tokens"][proper_bound[1]:])
                        data_to_link.append({
                            'id': f'{sub_section_index}-{entity_info["key"]}',
                            'label': 'unknown',
                            'label_id': -1,
                            'context_left': left_context.lower(), 
                            'mention': mention.lower(), 
                            'context_right': right_context.lower(),
                        })
    return data_to_link

In [ ]:
def augment_triple(triple_file_dict, nel_result, data_to_link):
    for sub_section_index, sub_section in enumerate(triple_file_dict):
        if "entities_info" in sub_section:
            for entity_info in sub_section["entities_info"]:
                entity_id = f'{sub_section_index}-{entity_info["key"]}',
                found_indexes = [i for i in range(len(data_to_link)) if data_to_link['id'] == entity_id]
                if (len(found_indexes) > 0):
                    found_index = found_indexes[0]
                    if (nel_result['scores'][found_index][0] > 0):
                        entity_info['entity_id'] = nel_result['predictions'][found_index][0]
    return triple_file_dict
                        
                        

In [ ]:
triple_files = glob2.glob(f"{triples_dir}/*.json")
pbar = tqdm(triple_files)
for triple_file in pbar:
    head, tail = os.path.split(triple_file)
    triple_file_dict = json.load(open(triple_file))
    data_to_link = process_triple(triple_file_dict)
    nel_result = blink_nel(data_to_link‍)
    aug_triple_file_dict = augment_triple(triple_file_dict, nel_result, data_to_link)
    save_dir = os.path.join(aug_triples_dir, tail)
    with open(save_dir, 'w') as f_handler:
        json.dump(aug_triple_file_dict, f_handler)
        